# Youtube Recommendation Video System

In [0]:
# Import Libraries
import pandas as pd
import numpy as np
import re
import time
import bs4 as bs4
import requests as rq
import json
import os 


 ## Data Search


* #### Webscraping youtube





## 1.  Youtube search page Structure (URL)




The image below shows the strucute of the youtube url. We have the main page, then the search page(opens when we search our query), the whay the videos will be sorted ( you define it by clickin on Filter) and finally the number of the page that's being searched. 


![alt text](https://drive.google.com/uc?id=15I0wM3OOiHQ3J3Ls_dDoq4Fax9PILFAw)



In [0]:
# what I want to search 
queries = ['machine+learnin','data+science','deploy' ] 
# The url structure of youtube - see image below
url = "http://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"

In [6]:
# example of a search query 
url.format(query ="machine+learning",page = 1)

'http://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=1'

In [0]:
# Making sure the folder I will store the raw data exists (create directory in case it does not exist)
data_dir = './raw_data'
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

In [20]:
for query in queries[:1]:
  for page in range(1,2):
    urll = url.format(query = query, page= page)
    print(urll)
    response = rq.get(urll) # get request to retrieve data from page

    with open("./raw_data/{}_{}.html".format(query,page),"w+") as output:
      output.write(response.text)
      time.sleep(2)



http://www.youtube.com/results?search_query=machine+learnin&sp=CAI%253D&p=1


# New Section